#**CONTENT_BASED FILTERING APPROACH TO BUILDING A RECOMMENDER SYSTEM**

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
rating_header = ['user_id', 'movie_id', 'user_rating', 'timestamp']
ratings = pd.read_csv('u.data', sep='\t', names= rating_header)

In [ ]:
ratings.head()

,user_id,movie_id,user_rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [ ]:
movie_header = ["movie_id",  "movie_title", "release_date", "video_release_date",
"IMDb_URL",  "unknown", "Action",  "Adventure", "Animation",
"Children's", "Comedy",  "Crime", "Documentary", "Drama", "Fantasy",
"Film-Noir", "Horror", "Musical", "Mystery", "Romance", "Sci-Fi",
"Thriller", "War", "Western" ]
filter_genres = ["movie_id", "unknown", "Action",  "Adventure", "Animation",
"Children's", "Comedy",  "Crime", "Documentary", "Drama", "Fantasy",
"Film-Noir", "Horror", "Musical", "Mystery", "Romance", "Sci-Fi",
"Thriller", "War", "Western" ]
movies_genre = pd.read_csv('u.item', sep='|', encoding='latin-1',names=movie_header)

In [ ]:
movies_genre.head()

,movie_id,movie_title,release_date,video_release_date,IMDb_URL,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0


In [ ]:
def f(row):
    movieid = row["movie_id"]
    title =["movie_title"] 
    genres = []
    if row["unknown"] ==1 :
      genres.append("unknown")
    if row["Action"]== 1:
      genres.append("Action")
    if row["Adventure"] == 1:
      genres.append("Adventure")
    if row ["Animation"] == 1:
      genres.append("Animation")
    if row ["Children's"] == 1:
      genres.append("Children's")
    if row ["Comedy"] == 1:
      genres.append("Comedy")
    if row ["Crime"] ==1 :
      genres.append("Crime")
    if row ["Documentary"] ==1 :
      genres.append("Documentary")
    if row ["Drama"] ==1:
      genres.append("Drama")
    if row ["Fantasy"] == 1:
      genres.append("Fantasy")
    if row ["Film-Noir"] == 1:
      genres.append("Film-Noir")
    if row ["Horror"] == 1:
      genres.append("Horror")
    if row ["Musical"] == 1:
      genres.append("Musical")
    if row ["Mystery"] == 1:
      genres.append("Mystery")
    if row ["Romance"] ==1:
      genres.append("Romance")
    if row ["Sci-Fi"] ==1 :
      genres.append("Sci-Fi")
    if row ["Thriller"] == 1:
      genres.append ("Thriller")
    if row ["War"] ==1:
      genres.append("War")
    if row ["Western"] == 1:
      genres.append("Western")

    return genres
                   

genre_col = movies_genre.drop(["release_date", "video_release_date",
"IMDb_URL" ], axis=1).apply(f,axis=1)
genre_df = movies_genre
genre_df["movie_genres"] = genre_col

In [ ]:
mv_genre = movies_genre[filter_genres]

In [ ]:
mv_genre.head()

,movie_id,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,4,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,5,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0


In [ ]:

def predict_top10(uid):
  # filter data for user with id 'uid'
  user_data = ratings[ratings["user_id"] == uid]
  user_movies = user_data.movie_id
  user_ratings = user_data.user_rating.values[np.argsort(user_movies)]
  user_movies = np.sort(user_movies)
  user_rated_movies = mv_genre.drop("movie_id", axis=1).values[user_movies,:]

  #simple test we are manipulating data correctly
  if (uid ==2):
    assert(user_ratings[user_movies == 314] == 1)
  n, m = user_rated_movies.shape
  assert(n == len(user_ratings))
  
  user_profile = user_rated_movies.T @ user_ratings
  w_user_profile = user_profile/sum(user_profile)
  genres = ["unknown", "Action",  "Adventure", "Animation",
"Children's", "Comedy",  "Crime", "Documentary", "Drama", "Fantasy",
"Film-Noir", "Horror", "Musical", "Mystery", "Romance", "Sci-Fi",
"Thriller", "War", "Western"]
  d = {'genre': genres, 'score': user_profile, 'weighted_score': w_user_profile}
  df = pd.DataFrame(data=d)
  df = df.sort_values(by= ["score"], ascending= False)
  print("\nUser profile for user %d is:\n"%(uid))
  print(df)

  # Now we use the user profile to find top 10 movie recommedations for the user
  # from movies we have not seen before
  sz, _ = mv_genre.shape
  test_movies = np.zeros(sz)
  test_movies[user_movies] = 1
  test_movies = (test_movies != 1)
  mv_test = mv_genre.drop("movie_id", axis=1).values[test_movies,:]
  #assert(mv_test.shape == ((sz - n+1), 19))
  mv_scores = mv_test@user_profile
  ord = np.argsort(-1*mv_scores)
  movies = mv_genre.movie_id.values[test_movies]
  movies = movies[ord]
  mv_scores_sorted = -1*np.sort(-1*mv_scores)

  # show the top 10 movie recommendations
  d2 = {'movie_id': movies, 'score': mv_scores_sorted}
  df2 = pd.DataFrame(data=d2)
  temp = genre_df[["movie_id", "movie_title", "movie_genres"]]
  df2 = pd.merge(df2, temp, how= "inner", on = "movie_id")
  print("\nTop 10 recommendations for user %d is:\n"%(uid))
  print(df2[0:10].drop(["movie_id", "score"], axis=1))
  return df, df2

  
  
  


In [ ]:
user_profile, top10 = predict_top10(2)



User profile for user 2 is:

          genre  score  weighted_score
8         Drama    122        0.250513
14      Romance     60        0.123203
16     Thriller     56        0.114990
5        Comedy     46        0.094456
1        Action     44        0.090349
6         Crime     30        0.061602
4    Children's     24        0.049281
15       Sci-Fi     22        0.045175
2     Adventure     22        0.045175
13      Mystery     16        0.032854
11       Horror     12        0.024641
17          War     10        0.020534
10    Film-Noir      7        0.014374
18      Western      5        0.010267
3     Animation      5        0.010267
9       Fantasy      3        0.006160
12      Musical      3        0.006160
7   Documentary      0        0.000000
0       unknown      0        0.000000

Top 10 recommendations for user 2 is:

                       movie_title                                      movie_genres
0                      Diva (1981)       [Action, Drama, Mystery,

In [ ]:
mv_genre.head(20)

,movie_id,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,4,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,5,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0
5,6,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
6,7,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0
7,8,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0
8,9,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
9,10,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0


#**ITEM_BASED FILTERING APPROACH TO BUILDING A RECOMMENDER SYSTEM**

In [ ]:
mv_rating = ratings.pivot(index = ["user_id"], values= ["user_rating"], columns = ["movie_id"])

In [ ]:
nn, mm = mv_rating.shape
denom = nn*mm
numer = sum(sum(np.isnan(mv_rating).values))
print("Sparsity of our data is:", numer/denom)

Sparsity of our data is: 0.9369533063577546


In [ ]:
#mv_rating.fillna(0).corr()
def norma(row):
  return (row -row.mean())/(row.max()-row.min())
qz=mv_rating.apply(norma)
qz = qz.fillna(0)
qz.shape

(943, 1682)

In [ ]:
def me(row):
  print(row.mean())

#qz.apply(me)
np.isnan(qz)

user_rating                       ...                            
movie_id        1      2      3      4     ...   1679   1680   1681   1682
user_id                                    ...                            
1              False  False  False  False  ...  False  False  False  False
2              False  False  False  False  ...  False  False  False  False
3              False  False  False  False  ...  False  False  False  False
4              False  False  False  False  ...  False  False  False  False
5              False  False  False  False  ...  False  False  False  False
...              ...    ...    ...    ...  ...    ...    ...    ...    ...
939            False  False  False  False  ...  False  False  False  False
940            False  False  False  False  ...  False  False  False  False
941            False  False  False  False  ...  False  False  False  False
942            False  False  False  False  ...  False  False  False  False
943            False  False  False  False  ...  False  False  False  False

[943 rows x 1682 columns]

In [ ]:
#Correlation matrix
corr_mat = qz.corr()

In [ ]:
def get_top10(movie_tt, rating = 5):
  good = -1
  if (rating >= 3):
    good=1

  z =movies_genre[movies_genre["movie_title"] == movie_tt].movie_id.values[0]
  vals =corr_mat.values[z-1]*good
  d2 = {'movie_id': np.arange(1, 1683), 'score': vals}
  df2 = pd.DataFrame(data=d2)
  temp = genre_df[["movie_id", "movie_title", "movie_genres"]]
  df2 = pd.merge(df2, temp, how= "inner", on = "movie_id")
  print("\nTop 10 recommendations for the movie %s is:\n"%(movie_tt))
  df2 = df2.sort_values(by= ["score"], ascending=False)
  print(df2[0:10].drop(["movie_id", "score"], axis=1))
  return df2




In [ ]:
nw= get_top10("Toy Story (1995)",5)


Top 10 recommendations for the movie Toy Story (1995) is:

                             movie_title                                       movie_genres
0                       Toy Story (1995)                    [Animation, Children's, Comedy]
587          Beauty and the Beast (1991)                   [Animation, Children's, Musical]
94                        Aladdin (1992)           [Animation, Children's, Comedy, Musical]
927                    Craft, The (1996)                                    [Drama, Horror]
425  Transformers: The Movie, The (1986)  [Action, Animation, Children's, Sci-Fi, Thrill...
70                 Lion King, The (1994)                   [Animation, Children's, Musical]
844            That Thing You Do! (1996)                                           [Comedy]
173       Raiders of the Lost Ark (1981)                                [Action, Adventure]
27                      Apollo 13 (1995)                          [Action, Drama, Thriller]
81                  

In [ ]:
corr_mat.values.shape

(1682, 1682)

In [ ]:
mv_genre

,movie_id,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,4,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,5,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,1678,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1678,1679,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0
1679,1680,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
1680,1681,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


##NEURAL NET BASED




In [ ]:
user_ids = ratings.user_id.unique().tolist()
user2user_encoded = {x: i for i, x in enumerate(user_ids)}
userencoded2user = {i: x for i, x in enumerate(user_ids)}
movie_ids = ratings.movie_id.unique().tolist()
movie2movie_encoded = {x: i for i, x in enumerate(movie_ids)}
movie_encoded2movie = {i: x for i, x in enumerate(movie_ids)}
dfz = pd.DataFrame()
dfz["user"] = ratings.user_id.map(user2user_encoded)
dfz["movie"] = ratings.movie_id.map(movie2movie_encoded)

num_users = len(user2user_encoded)
num_movies = len(movie_encoded2movie)
dfz["rating"] = ratings.user_rating.values.astype(np.float32)
# min and max ratings will be used to normalize the ratings later
min_rating = min(dfz["rating"])
max_rating = max(dfz["rating"])

print(
    "Number of users: {}, Number of Movies: {}, Min rating: {}, Max rating: {}".format(
        num_users, num_movies, min_rating, max_rating
    )
)

Number of users: 943, Number of Movies: 1682, Min rating: 1.0, Max rating: 5.0


In [ ]:
from sklearn.model_selection import train_test_split

y =dfz["rating"].apply(lambda x: (x - min_rating) / (max_rating - min_rating))
y = y.values
X = dfz [["user","movie"]].values
xtrain, xtest,ytrain, ytest = train_test_split(X,y ,test_size=0.2)

In [ ]:
y

array([0.5 , 0.5 , 0.  , ..., 0.  , 0.25, 0.5 ])

In [ ]:
EMBEDDING_SIZE = 50

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
class RecommenderNet(keras.Model):
    def __init__(self, num_users, num_movies, embedding_size, **kwargs):
        super(RecommenderNet, self).__init__(**kwargs)
        self.num_users = num_users
        self.num_movies = num_movies
        self.embedding_size = embedding_size
        self.user_embedding = layers.Embedding(
            num_users,
            embedding_size,
            embeddings_initializer="he_normal",
            embeddings_regularizer=keras.regularizers.l2(1e-6),
        )
        self.user_bias = layers.Embedding(num_users, 1)
        self.movie_embedding = layers.Embedding(
            num_movies,
            embedding_size,
            embeddings_initializer="he_normal",
            embeddings_regularizer=keras.regularizers.l2(1e-6),
        )
        self.movie_bias = layers.Embedding(num_movies, 1)

    def call(self, inputs):
        user_vector = self.user_embedding(inputs[:, 0])
        user_bias = self.user_bias(inputs[:, 0])
        movie_vector = self.movie_embedding(inputs[:, 1])
        movie_bias = self.movie_bias(inputs[:, 1])
        dot_user_movie = tf.tensordot(user_vector, movie_vector, 2)
        # Add all the components (including bias)
        x = dot_user_movie + user_bias + movie_bias
        # The sigmoid activation forces the rating to between 0 and 1
        return tf.nn.sigmoid(x)


model = RecommenderNet(num_users, num_movies, EMBEDDING_SIZE)
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(), optimizer=keras.optimizers.Adam(lr=0.001)
)

In [ ]:
history = model.fit(
    x=xtrain,
    y=ytrain,
    batch_size=64,
    epochs=7,
    verbose=1,
    validation_data=(xtest, ytest),
)

Epoch 1/7
1250/1250 [==============================] - 2s 2ms/step - loss: 0.6542 - val_loss: 0.6336
Epoch 2/7
1250/1250 [==============================] - 2s 2ms/step - loss: 0.6304 - val_loss: 0.6272
Epoch 3/7
1250/1250 [==============================] - 2s 2ms/step - loss: 0.6215 - val_loss: 0.6187
Epoch 4/7
1250/1250 [==============================] - 2s 2ms/step - loss: 0.6186 - val_loss: 0.6179
Epoch 5/7
1250/1250 [==============================] - 2s 2ms/step - loss: 0.6174 - val_loss: 0.6171
Epoch 6/7
1250/1250 [==============================] - 2s 2ms/step - loss: 0.6168 - val_loss: 0.6168
Epoch 7/7
1250/1250 [==============================] - 2s 2ms/step - loss: 0.6157 - val_loss: 0.6160


In [ ]:
from keras.models import Model
from keras.layers import Input, Reshape, Dot
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.layers import Lambda
import keras.backend as K


In [ ]:
class EmbeddingLayer:
    def __init__(self, n_items, n_factors):
        self.n_items = n_items
        self.n_factors = n_factors
    
    def __call__(self, x):
        x = Embedding(self.n_items, self.n_factors, embeddings_initializer='he_normal',
                      embeddings_regularizer=l2(1e-6))(x)
        x = Reshape((self.n_factors,))(x)
        return x

from keras.layers import Concatenate, Dense, Dropout
def RecommenderNetz(n_users, n_movies, n_factors, min_rating, max_rating):
    usr = Input(shape= (1, ))
    u = Embedding(n_users, n_factors, embeddings_initializer='he_normal',
                      embeddings_regularizer=l2(1e-6) )(usr)
    u = Reshape((n_factors,))(u)
    
    mv = Input(shape=(1, ))
    m = Embedding(n_movies, n_factors, embeddings_initializer='he_normal', embeddings_regularizer =l2(1e-6))(mv)
    m = Reshape((n_factors, ))(m)

    x = Concatenate()([u, m])
    x = Dropout(0.42)(x)
    x = Dense(12, kernel_initializer='he_normal', activation= "relu")(x)
    x = Dropout(0.42)(x)
    x = Dense(5, kernel_initializer='he_normal', activation= "relu")(x)   
    x = Dropout(0.42)(x)
    x = Dense(1, kernel_initializer='he_normal', activation="sigmoid")(x)
    model = Model(inputs=[usr, mv], outputs=x)
    model.compile(loss= 'mean_squared_error', optimizer=Adam(lr =0.001))
    return model
  

In [ ]:
model3 = RecommenderNetz(num_users, num_movies, 50, min_rating, max_rating)
model3.summary()

Model: "functional_17"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_19 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_20 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_22 (Embedding)        (None, 1, 50)        47150       input_19[0][0]                   
__________________________________________________________________________________________________
embedding_23 (Embedding)        (None, 1, 50)        84100       input_20[0][0]                   
______________________________________________________________________________________

In [ ]:
history = model3.fit(x=xtrain, y=ytrain, batch_size=64, epochs=7, validation_data=(xtest, ytest))

Epoch 1/7


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:432: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


1250/1250 [==============================] - 3s 2ms/step - loss: 0.0732 - val_loss: 0.0619
Epoch 2/7
1250/1250 [==============================] - 3s 2ms/step - loss: 0.0662 - val_loss: 0.0586
Epoch 3/7
1250/1250 [==============================] - 3s 2ms/step - loss: 0.0638 - val_loss: 0.0573
Epoch 4/7
1250/1250 [==============================] - 3s 2ms/step - loss: 0.0625 - val_loss: 0.0567
Epoch 5/7
1250/1250 [==============================] - 3s 2ms/step - loss: 0.0618 - val_loss: 0.0563
Epoch 6/7
1250/1250 [==============================] - 3s 2ms/step - loss: 0.0614 - val_loss: 0.0562
Epoch 7/7
1250/1250 [==============================] - 3s 2ms/step - loss: 0.0612 - val_loss: 0.0566


In [ ]:
from keras.layers import Add, Activation, Lambda
from keras.layers import Concatenate, Dense, Dropout
def RecommenderNet2(n_users, n_movies, n_factors, min_rating, max_rating):

    user = Input(shape=(1))
    print("I AM HERE", user.shape)
    u = EmbeddingLayer(n_users, n_factors)(user)
    
    movie = Input(shape=(1,))
    m = EmbeddingLayer(n_movies, n_factors)(movie)
    
    x = Concatenate()([u, m])
    x = Dropout(0.05)(x)
    
    x = Dense(10, kernel_initializer='he_normal')(x)
    x = Activation('relu')(x)
    x = Dropout(0.5)(x)
    
    x = Dense(1, kernel_initializer='he_normal')(x)
    x = Activation('sigmoid')(x)
    x = Lambda(lambda x: x * (max_rating - min_rating) + min_rating)(x)
    model = Model(inputs=[user, movie], outputs=x)
    opt = Adam(lr=0.001)
    model.compile(loss='mean_squared_error', optimizer=opt)
    return model

In [ ]:
model = tf.keras.Sequential()


In [ ]:
from keras.models import Model
from keras.layers import Input, Reshape, Dot
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.regularizers import l2
def RecommenderV1(n_users, n_movies, n_factors):
    user = Input(shape=(1,))
    u = Embedding(n_users, n_factors, embeddings_initializer='he_normal',
                  embeddings_regularizer=l2(1e-6))(user)
    u = Reshape((n_factors,))(u)
    
    movie = Input(shape=(1,))
    m = Embedding(n_movies, n_factors, embeddings_initializer='he_normal',
                  embeddings_regularizer=l2(1e-6))(movie)
    m = Reshape((n_factors,))(m)
    
    x = Dot(axes=1)([u, m])
    model = Model(inputs=[user, movie], outputs=x)
    opt = Adam(lr=0.001)
    model.compile(loss='mean_squared_error', optimizer=opt)
    return model

In [ ]:
import matplotlib.pyplot as plt


In [ ]:
xtrain =[xtrain[:, 0], xtrain[:, 1]]
xtest = [xtest[:, 0], xtest[:, 1]]

In [ ]:
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.title("model loss")
plt.ylabel("loss")
plt.xlabel("epoch")
plt.legend(["train", "test"], loc="upper left")
plt.show()

In [ ]:
model.predict([1, 2])

In [ ]:
user_id = ratings.user_id.sample(1).iloc[0]
movies_watched_by_user = dfz[dfz.user == user_id]
movies_not_watched = movie_df[
    ~movie_df["movieId"].isin(movies_watched_by_user.movieId.values)
]["movieId"]
movies_not_watched = list(
    set(movies_not_watched).intersection(set(movie2movie_encoded.keys()))
)
movies_not_watched = [[movie2movie_encoded.get(x)] for x in movies_not_watched]
user_encoder = user2user_encoded.get(user_id)
user_movie_array = np.hstack(
    ([[user_encoder]] * len(movies_not_watched), movies_not_watched)
)
ratings = model.predict(user_movie_array).flatten()
top_ratings_indices = ratings.argsort()[-10:][::-1]
recommended_movie_ids = [
    movie_encoded2movie.get(movies_not_watched[x][0]) for x in top_ratings_indices
]

NameError: ignored

In [ ]:
import pandas as pd
import numpy as np
from zipfile import ZipFile
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from pathlib import Path
import matplotlib.pyplot as plt

# Download the actual data from http://files.grouplens.org/datasets/movielens/ml-latest-small.zip"
# Use the ratings.csv file
movielens_data_file_url = (
    "http://files.grouplens.org/datasets/movielens/ml-100k.zip"
)
movielens_zipped_file = keras.utils.get_file(
    "ml-latest-small.zip", movielens_data_file_url, extract=False
)
keras_datasets_path = Path(movielens_zipped_file).parents[0]
movielens_dir = keras_datasets_path / "ml-latest-small"

# Only extract the data the first time the script is run.
if not movielens_dir.exists():
    with ZipFile(movielens_zipped_file, "r") as zip:
        # Extract files
        print("Extracting all the files now...")
        zip.extractall(path=keras_datasets_path)
        print("Done!")

ratings_file = movielens_dir / "ratings.csv"
df = pd.read_csv(ratings_file)

In [ ]:
df

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [ ]:
xtrain

[array([896, 311, 156, ..., 184, 142, 252]),
 array([ 12, 640,  60, ..., 503, 700, 432])]

In [ ]:
xtest

[array([646, 227,  39, ..., 556, 649,  38]),
 array([666,  98, 290, ..., 794, 221, 104])]

In [ ]:
xtrain[0]

array([896, 311, 156, ..., 184, 142, 252])